The goal of this notebook is to play around with how time information is used in unet in controlnet.

---

In [1]:
import torch
import torch.nn as nn

In [2]:
device = "cpu"
device_dtype = torch.float32

In [3]:
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel

In [4]:
controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=device_dtype)

In [5]:
pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5", controlnet=controlnet, torch_dtype=device_dtype
).to(device)

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


In [7]:
unet = pipe.unet

In [24]:
def public_attrs(o, contains=''): return [a for a in dir(o) if not a.startswith('_') and contains in a]

In [26]:
public_attrs(unet, contains='time')

['time_embed_act', 'time_embedding', 'time_proj']

In [32]:
unet.time_embedding

TimestepEmbedding(
  (linear_1): Linear(in_features=320, out_features=1280, bias=True)
  (act): SiLU()
  (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
)

In [28]:
unet.time_proj

Timesteps()

In [30]:
unet.time_embed_act??

Type:        NoneType
String form: None
Docstring:   <no docstring>

**Q1**: I assumed `time_proj` to be a module, but it is of type `Timesteps`. Could it actually be the embedding, and not the projection?

**Q2**: I assumed `time_embedding` to the time embedding (ie a deterministic, non-parametric function), but is is a module. 

**Q3**: Are the names just swapped, ie diffusers calls 'projection' what I understand as 'embedding', and vice versa?

In [34]:
unet.time_proj??

Signature:       unet.time_proj(*args, **kwargs)
Type:            Timesteps
String form:     Timesteps()
File:            ~/Documents/GitHub/diffusers/diffusers/src/diffusers/models/embeddings.py
Source:         
class Timesteps(nn.Module):
    def __init__(self, num_channels: int, flip_sin_to_cos: bool, downscale_freq_shift: float):
        super().__init__()
        self.num_channels = num_channels
        self.flip_sin_to_cos = flip_sin_to_cos
        self.downscale_freq_shift = downscale_freq_shift

    def forward(self, timesteps):
        t_emb = get_timestep_embedding(
            timesteps,
            self.num_channels,
            flip_sin_to_cos=self.flip_sin_to_cos,
            downscale_freq_shift=self.downscale_freq_shift,
        )
        return t_emb
Class docstring:
Base class for all neural network modules.

Your models should also subclass this class.

Modules can also contain other Modules, allowing to nest them in
a tree structure. You can assign the submodules as

`Timesteps` (the type of `time_proj`) has a forward method that only applies `get_timestep_embedding`. Let's look at that.

In [36]:
from diffusers.models.embeddings import get_timestep_embedding

In [37]:
get_timestep_embedding??

Signature:
get_timestep_embedding(
    timesteps: torch.Tensor,
    embedding_dim: int,
    flip_sin_to_cos: bool = False,
    downscale_freq_shift: float = 1,
    scale: float = 1,
    max_period: int = 10000,
)
Source:   
def get_timestep_embedding(
    timesteps: torch.Tensor,
    embedding_dim: int,
    flip_sin_to_cos: bool = False,
    downscale_freq_shift: float = 1,
    scale: float = 1,
    max_period: int = 10000,
):
    """
    This matches the implementation in Denoising Diffusion Probabilistic Models: Create sinusoidal timestep embeddings.

    :param timesteps: a 1-D Tensor of N indices, one per batch element.
                      These may be fractional.
    :param embedding_dim: the dimension of the output. :param max_period: controls the minimum frequency of the
    embeddings. :return: an [N x dim] Tensor of positional embeddings.
    """
    assert len(timesteps.shape) == 1, "Timesteps should be a 1d-array"

    half_dim = embedding_dim // 2
    exponent = -math.log

**A1: Yes,** `time_proj` is actually the embedding! Weird naming, but okay.

___

Now, let's look at `TimestepEmbedding`, which is the type of `time_embedding`.

In [39]:
from diffusers.models.embeddings import TimestepEmbedding

In [44]:
TimestepEmbedding.forward??

Signature: TimestepEmbedding.forward(self, sample, condition=None)
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, sample, condition=None):
        if condition is not None:
            sample = sample + self.cond_proj(condition)
        sample = self.linear_1(sample)

        if self.act is not None:
            sample = self.act(sample)

        sample = self.linear_2(sample)

        if self.post_act is not None:
            sample = self.post_act(sample)
        return sample
File:      ~/Documents/GitHub/diffusers/diffusers/src/diffusers/models/embeddings.py
Type:      function

**A2: Yes,** `time_embedding` is what I understand as a projection.

**A3: Yes, the names for these concepts are swapped!**